In [24]:
import pprint
import time

import ee

ee.Initialize()


### Export Parameters

In [34]:
scene_coll_id = 'projects/earthengine-legacy/assets/projects/openet/ptjpl/landsat/scene'

start_date = '2017-07-01'
end_date = '2017-08-01'

wrs2_tiles = ['p042r035']
# filter_geom = ee.Geometry.Point(-119.5, 36.0)

gdrive_folder = 'EE_Exports'

# This code only supports exporting a single band (since PTJPL only has one band)
variable = 'et'


### Build the scene collection you wish to download

In [35]:
# Build the filtered collection
scene_coll = ee.ImageCollection(scene_coll_id) \
    .filterDate(start_date, end_date)

# Filter the collection by WRS2 path/row and/or geometry
if wrs2_tiles:
    scene_coll = scene_coll.filter(ee.Filter.inList('wrs2_tile', wrs2_tiles))
# if filter_geom:
#     scene_coll = scene_coll.filterBounds(filter_geom)
    
# The only band available is ET so it isn't necessary to select it
#     .select([variable])
    
scene_id_list = scene_coll.aggregate_array('system:index').getInfo()
    
# Print the scene IDs of the filtered collection
pprint.pprint(scene_id_list)


['lc08_042035_20170702',
 'lc08_042035_20170718',
 'le07_042035_20170710',
 'le07_042035_20170726']


### Start the exports

In [28]:
for scene_id in scene_id_list:
    print(f'{scene_id}')
    image_id = f'{scene_coll_id}/{scene_id}'
    export_id = f'ptjpl_{variable}_{scene_id}'
    
    # Get the projection, shape and transform from the image
    output_img = ee.Image(image_id)
    output_info = output_img.getInfo()
    # output_proj = output_img.projection().getInfo()
    # pprint.pprint(output_info)
    # pprint.pprint(output_proj)
    
    # Apply the scale factor if necessary
    # The casting probably isn't necessary but I was just playing around
    if ('scale_factor' in output_info['properties'].keys() and 
           output_info['properties']['scale_factor'] != 1):
        output_img = output_img \
            .toDouble() \
            .multiply(output_info['properties']['scale_factor'])
        #     .toFloat()
    
    # Build the export task
    task = ee.batch.Export.image.toDrive(
        output_img,
        description=export_id,
        folder=gdrive_folder,
        fileNamePrefix=export_id,
        dimensions=output_info['bands'][0]['dimensions'],
        crs=output_info['bands'][0]['crs'],
        crsTransform=output_info['bands'][0]['crs_transform'],
        # maxPixels=1E10,
        # pyramidingPolicy='mean',
    )
    
    # Start the export task
    task.start()
    
    time.sleep(1)
    # pprint.pprint(task.status())


lc08_042035_20170702
{'creation_timestamp_ms': 1590540124443,
 'description': 'ptjpl_et_lc08_042035_20170702',
 'id': 'SXKTV5ZK6LM34TWZACTY2CAL',
 'name': 'projects/earthengine-legacy/operations/SXKTV5ZK6LM34TWZACTY2CAL',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1590540124443}
lc08_042035_20170718
{'creation_timestamp_ms': 1590540126624,
 'description': 'ptjpl_et_lc08_042035_20170718',
 'id': 'HTITFXKFUXTJBPXYPQAUVAGT',
 'name': 'projects/earthengine-legacy/operations/HTITFXKFUXTJBPXYPQAUVAGT',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1590540126624}
le07_042035_20170710
{'creation_timestamp_ms': 1590540128777,
 'description': 'ptjpl_et_le07_042035_20170710',
 'id': '6L6IQN6E77DURNJSYW322OXF',
 'name': 'projects/earthengine-legacy/operations/6L6IQN6E77DURNJSYW322OXF',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1590540